According to the Lab Handout;

Rewrite version below

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

class Problem(object):
  def __init__(self, initial=None, goal=None, **kwds):
    self.__dict__.update(initial=initial, goal=goal, **kwds)
  def actions(self, state): raise NotImplementedError
  def result(self, state, action): raise NotImplementedError
  def is_goal(self, state): return state == self.goal
  def action_cost(self, s, a, s1): return 1
  def h(self, node): return 0

  def __str__(self):
    return '{}({!r}, {!r})'.format(
      type(self).__name__, self.initial, self.goal)

In [ ]:
class Node:
  "A Node in a search tree."
  def __init__(self, state, parent=None, action=None, path_cost=0):
    self.__dict__.update(state=state, parent=parent, action=action,
path_cost=path_cost)
    
  def __repr__(self): return '<{}>'.format(self.state)
  def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
  def __lt__(self, other): return self.path_cost < other.path_cost

failure = Node('failure', path_cost=math.inf)
cutoff = Node('cutoff', path_cost=math.inf)

def expand(problem, node):
  "Expand a node, generating the children nodes."
  s = node.state
  for action in problem.actions(s):
    s1 = problem.result(s, action)
    cost = node.path_cost + problem.action_cost(s, action, s1)
    yield Node(s1, node, action, cost)

def path_actions(node):
  "The sequence of actions to get to this node."
  if node.parent is None:
    return []
  return path_actions(node.parent) + [node.action]

def path_states(node):
  "The sequence of states to get to this node."
  if node in (cutoff, failure, None):
    return []
  return path_states(node.parent) + [node.state]

In [ ]:
FIFOQueue = deque
LIFOQueue = list

class PriorityQueue:

  def __init__(self, items=(), key=lambda x: x):
    self.key = key
    self.items = [] # a heap of (score, item) pairs
    for item in items:
      self.add(item)

  def add(self, item):
    pair = (self.key(item), item)
    heapq.heappush(self.items, pair)

  def pop(self):
    """Pop and return the item with min f(item) value."""
    return heapq.heappop(self.items)[1]

  def top(self):
    return self.items[0][1]

  def __len__(self):
    return len(self.items)

In [ ]:
def best_first_search(problem, f):
  "Search nodes with minimum f(node) value first."
  node = Node(problem.initial)
  frontier = PriorityQueue([node], key=f)
  reached = {problem.initial: node}
  while frontier:
    node = frontier.pop()
    if problem.is_goal(node.state):
      return node

    for child in expand(problem, node):
      s = child.state
      if s not in reached or child.path_cost < reached[s].path_cost:
        reached[s] = child
        frontier.add(child)
  return failure

def g(n): return n.path_cost

def depth_first_search(problem):
  "Search deepest nodes in the search tree first; using best-first."
  return best_first_search(problem, f=lambda n: -len(n))

def breadth_first_search(problem):
  "Search shallowest nodes in the search tree first; using best-first."
  return best_first_search(problem, f=len)

def uniform_cost_search(problem):
  "Search nodes with minimum path cost first."
  return best_first_search(problem, f=g)

def a_search(problem):
  "Straight-line distance between state and the goal."
  return best_first_search(problem, f=lambda n: g(n) + problem.h(n))

In [ ]:
class RouteProblem(Problem):

  def actions(self, state):
    """The places neighboring `state`."""
    return self.map.neighbors[state]
    
  def result(self, state, action):
    """Go to the `action` place, if the map says that is possible."""
    return action if action in self.map.neighbors[state] else state

  def action_cost(self, s, action, s1):
    """The distance (cost) to go from s to s1."""
    return self.map.distances[s, s1]

  def h(self, node):
    "Straight-line distance between state and the goal."
    locs = self.map.locations
    return straight_line_distance(locs[node.state], locs[self.goal])

def straight_line_distance(A, B):
  "Straight-line distance between two points."
  return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

class Map:
  def __init__(self, links, locations=None, directed=False):
    if not hasattr(links, 'items'): # Distances are 1 by default
      links = {link: 1 for link in links}
    if not directed:
      for (v1, v2) in list(links):
        links[v2, v1] = links[v1, v2]
    self.distances = links
    self.neighbors = multimap(links)
    self.locations = locations or defaultdict(lambda: (0, 0))

def multimap(pairs) -> dict:
  "Given (key, val) pairs, make a dict of {key: [val,...]}."
  result = defaultdict(list)
  for key, val in pairs:
    result[key].append(val)
  return result

In [ ]:
romania = Map(
{('O', 'Z'): 71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'):
118,
('L', 'T'): 111, ('L', 'M'): 70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'):
146,
('C', 'P'): 138, ('R', 'S'): 80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'):
101,
('B', 'G'): 90, ('B', 'U'): 85, ('H', 'U'): 98, ('E', 'H'): 86, ('U', 'V'):
142,
('I', 'V'): 92, ('I', 'N'): 87, ('P', 'R'): 97},
{'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558,
294),
'F': (285, 460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162,
379),
'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227,
412),
'S': (187, 463), 'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92,
539)})

r1 = RouteProblem('A', 'C', map=romania)

print("Lowest-cost path from Arad to Criova")
print(path_states(uniform_cost_search(r1)))
print("BFS")
print(path_states(breadth_first_search(r1)))
print("DFS")
print(path_states(depth_first_search(r1)))
print("A* Search")
print(path_states(a_search(r1)))

Lowest-cost path from Arad to Criova
['A', 'S', 'R', 'C']
BFS
['A', 'S', 'R', 'C']
DFS
['A', 'T', 'L', 'M', 'D', 'C']
A* Search
['A', 'S', 'R', 'C']


Rewrite the Function

In [ ]:
def dfs_re(problem):
  node = Node(problem.initial)
  frontier = LIFOQueue([node])
  reached = {problem.initial: node}

  while frontier:
    node = frontier.pop()
    if problem.is_goal(node.state):
      return node

    for child in expand(problem, node):
      s = child.state
      if s not in reached or child.path_cost < reached[s].path_cost:
        reached[s] = child
        frontier.append(child)

  return failure


def bfs_re(problem):
  node = Node(problem.initial)
  frontier = FIFOQueue([node])
  reached = {problem.initial: node}

  while frontier:
    node = frontier.pop()
    if problem.is_goal(node.state):
      return node

    for child in expand(problem, node):
      s = child.state
      if s not in reached or child.path_cost < reached[s].path_cost:
        reached[s] = child
        frontier.appendleft(child)

  return failure


def ucs_re(problem):
  node = Node(problem.initial)
  frontier = PriorityQueue([node], key=g)
  reached = {problem.initial: node}

  while frontier:
    node = frontier.pop()
    if problem.is_goal(node.state):
      return node

    for child in expand(problem, node):
      s = child.state
      if s not in reached or child.path_cost < reached[s].path_cost:
        reached[s] = child
        frontier.add(child)
        
  return failure

def a_re(problem):
  node = Node(problem.initial)
  frontier = PriorityQueue([node], key=lambda n: g(n) + problem.h(n))
  reached = {problem.initial: node}

  while frontier:
    node = frontier.pop()
    if problem.is_goal(node.state):
      return node

    for child in expand(problem, node):
      s = child.state
      if s not in reached or child.path_cost < reached[s].path_cost:
        reached[s] = child
        frontier.add(child)

In [ ]:
print("Jamie Juntorn Thiantanukij")
print("2021610213")
print("")

print("Rewrite the function Uniform Cost Search")
print("Answer:")
print(path_states(ucs_re(r1)))
print("")

print("Rewrite the function Breadth-First Search")
print("Answer:")
print(path_states(bfs_re(r1)))
print("")

print("Rewrite the function Depth-First Search")
print("Answer:")
print(path_states(dfs_re(r1)))
print("")

print("Rewrite the function A* Search")
print("Answer:")
print(path_states(a_re(r1)))

Jamie Juntorn Thiantanukij
2021610213

Rewrite the function Uniform Cost Search
Answer:
['A', 'S', 'R', 'C']

Rewrite the function Breadth-First Search
Answer:
['A', 'S', 'R', 'C']

Rewrite the function Depth-First Search
Answer:
['A', 'T', 'L', 'M', 'D', 'C']

Rewrite the function A* Search
Answer:
['A', 'S', 'R', 'C']


In [ ]:
#For Hand Calculation

def generate_dist_to_c():
  locate = {'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558, 294),
            'F': (285,460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162, 379),
            'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227, 412),
            'S': (187, 463),'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92, 539)}
  for i in locate:
    print("Distance from {} to C : {}".format(i, straight_line_distance(locate[i], locate['C'])))

generate_dist_to_c()

Distance from A to C : 271.7425251961864
Distance from B to C : 159.62455951387932
Distance from C to C : 0.0
Distance from D to C : 86.70063436907483
Distance from E to C : 312.1297807002722
Distance from F to C : 179.2930561957155
Distance from G to C : 125.1718818265508
Distance from H to C : 310.00645154577023
Distance from I to C : 347.9425239892359
Distance from L to C : 126.0634760745554
Distance from M to C : 103.73041983911952
Distance from N to C : 319.5262117573455
Distance from O to C : 321.97204847626136
Distance from P to C : 108.60018416190647
Distance from R to C : 128.41339493993607
Distance from S to C : 187.52333188166213
Distance from T to C : 207.87015177749788
Distance from U to C : 238.1365154695936
Distance from V to C : 344.7680379617577
Distance from Z to C : 297.03871801500895
